<a href="https://colab.research.google.com/github/diana-bsv/rag-system/blob/main/RAG_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch

from sklearn.neighbors import NearestNeighbors

from transformers import AutoTokenizer, AutoModel, DPRContextEncoder, pipeline

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Model for encoding the context
tokenizer = AutoTokenizer.from_pretrained("facebook/dpr-ctx_encoder-multiset-base", device = device)
model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")


In [ ]:
# Model for encoding the question
q_tokenizer = AutoTokenizer.from_pretrained("facebook/dpr-question_encoder-multiset-base", device = device)
q_model = AutoModel.from_pretrained("facebook/dpr-question_encoder-multiset-base")

In [ ]:
# Model for context question answering
pipe = pipeline("text2text-generation", model="google/flan-t5-base", device = device)

In [6]:
def make_chunks(path):
  f = open(path, "r")
  data = f.read()
  f.close()

  words = data.split()

  chunks = []

  for i in range(0, len(words), 100):
    tmp = " ".join(words[i:i+100])

    tmp.strip()

    if tmp:
      chunks.append(tmp)

  print(f"Number of chunks: {len(chunks)}")
  return chunks

In [7]:
def vectorize_chunks(chunks, tokenizer, model):

  tokenized = tokenizer(chunks, add_special_tokens=True, return_token_type_ids=False, padding=True)


  input_ids = torch.tensor(tokenized.input_ids)
  attention_mask = torch.tensor(tokenized.attention_mask)

  model = model.to(device)

  last_hidden_states = []
  with torch.no_grad():
    for i in tqdm(range(0, len(input_ids), 100)):
      tmp1 = input_ids[i:i+100].to(device)
      tmp2 = attention_mask[i:i+100].to(device)
      last_hidden_states.append(model(tmp1, attention_mask=tmp2).pooler_output.detach().cpu())


  features = torch.cat(last_hidden_states, dim=0)
  print(f"Done\nShape of features array: {features.shape}")

  return features


In [8]:
def vectorize_question(question, q_tokenizer, q_model):
  q_tokenized = q_tokenizer(question, add_special_tokens=True, return_token_type_ids=False)
  id = torch.tensor([q_tokenized.input_ids])
  mask = torch.tensor([q_tokenized.attention_mask])

  outp = q_model(id, attention_mask=mask).pooler_output.detach()

  return outp[0]


In [9]:
def find_nearest_chunks(features, question, n_neighbors=5):

  features = features.numpy()

  nn_model = NearestNeighbors(n_neighbors=n_neighbors, metric="cosine")
  nn_model.fit(features)

  target_vector = question.reshape(1, -1)

  distances, indices = nn_model.kneighbors(target_vector)

  # print(f"Closest chunks indices {indices}")
  # print()
  # print(f"Distanses {distances}")

  return indices[0].tolist()

In [10]:
def answer(question, chunks, features, pipe, NN, q_tokenizer, q_model):

  question_vector = vectorize_question(question, q_tokenizer, q_model)
  indices = find_nearest_chunks(features, question_vector, n_neighbors=NN)

  relevant = [chunks[i] for i in indices]

  prompt = "Source: " + " ".join(relevant[:5]) + "\n\nUsing the source answer the question: " + question


  answer = pipe(prompt)[0]["generated_text"]

  print(f"Q: {question}")
  print(f"A: {answer}")
  print()
  print("Based on:")

  for line in relevant:
    print(line)


In [ ]:
PATH = "./Looking-for-Alaska.txt"
NN = 3 # Number of nearest neighbours

chunks = make_chunks(PATH)

features = vectorize_chunks(chunks, tokenizer, model)

In [12]:
#True
question = "How to get out of the labyrinth?"

answer(question, chunks, features, pipe, NN, q_tokenizer, q_model)

Q: How to get out of the labyrinth?
A: straight and fast

Based on:
at that moment reaching the finish line. The rest was darkness. "Damn it," he sighed. "How will I ever get out of this labyrinth!" The whole passage was underlined in bleeding, water-soaked black ink. But there was another ink, this one a crisp blue, post-flood, and an arrow led from "How will I ever get out of this labyrinth!" to a margin note written in her loop-heavy cursive: Straight &amp; Fast. "Hey, she wrote something in here after the flood," I said. "But it's weird. Look. Page one ninety-two." I tossed the book to the Colonel, and he flipped to
put my soul to rest." "Or hers," I said. "Right. I'd forgotten about her." He shook his head. "That keeps happening." "Well, you have to write something,"I argued. "After all this time, it still seems to me like straight and fast is the only way out — but I choose the labyrinth. The labyrinth blows, but I choose it." one hundred thirty-six days after Two weeks later,I s

In [13]:
#Wrong
question = "Who is Alaska?"

answer(question, chunks, features, pipe, NN, q_tokenizer, q_model)

Q: Who is Alaska?
A: Flow Reeda

Based on:
were me, you, and Jake," I said to him. "And we don't know. So how the hell are we going to find out?" Takumi looked over at the Colonel and sighed. "I don't think it would help, to know where she was going. I think that would make it worse for us. Just a gut feeling." "Well, mygut wants to know," Lara said, and only then did I realize what Takumi meant the day we'd showered together — I may have kissed her, but I really didn'thave a monopoly on Alaska; the Colonel and I weren't the only ones who
kids she did not consider Weekday Warriors and piled us into her tiny blue two-door. By happy coincidence, a cute sophomore named Lara ended up sitting on my lap. Lara'd been born in Russia or someplace, and she spoke with a slight accent. Since we were only four layers of clothes from doing it, I took the opportunity to introduce myself. "I know who you are." She smiled. "You're Alaska's freend from Flow Reeda." "Yup. Get ready for a lot of dumb ques

In [14]:

#True
question = "Where is Colonel from?"

answer(question, chunks, features, pipe, NN, q_tokenizer, q_model)

Q: Where is Colonel from?
A: New Hope, Alabama

Based on:
"Uh, no. Whatever is fine." "I see you've decorated the place," he said, gesturing toward the world map. "I like it." And then he started naming countries. He spoke in a monotone, as if he'd done it a thousand times before. Afghanistan. Albania. Algeria. American Samoa. Andorra. And so on. He got through the A'sbefore looking up and noticing my incredulous stare. "I could do the rest, but it'd probably bore you. Something I learned over the summer. God, you can't imagine how boring New Hope, Alabama, is in the summertime. Like watching soybeans grow. Where are you from, by
Colonel behind the wheel of Takumi's SUV. We asked Lara and Takumi to come along, but they were tired of chasing ghosts, and besides, finals were coming. It was a bright afternoon, and the sun bore down on the asphalt so that the ribbon of road before us quivered with heat. We drove a mile down Highway 119 and then merged onto I-65 northbound, heading toward t

In [15]:
#True
question = "Who are Colonel's parents?"

answer(question, chunks, features, pipe, NN, q_tokenizer, q_model)

Q: Who are Colonel's parents?
A: Dolores

Based on:
embarrassed of his mom at all. He was just scared that we would act like condescending boarding-school snobs. I'd always found the Colonel's I-hate-the-rich routine a little overwrought until I saw him with his mom. He was the same Colonel, but in a totally different context. It made me hope that one day, I could meet Alaska's family, too. Dolores insisted that Alaska and I share the bed, and she slept on the pull-out while the Colonel was out in his tent. I worried he would get cold, but frankly I wasn't about to give up my bed with Alaska.
Colonel was short — he couldn't afford to be any taller. The place was really one long room, with a full-size bed in the front, a kitchenette, and a living area in the back with a TV and a small bathroom — so small that in order to take a shower, you pretty much had to sit on the toilet. "It ain't much," the Colonel's mom ("That's Dolores, not Miss Martin") told us. "But y'alls a-gonna have a turk

In [16]:
#True
question = "What is the сapital of Uzbekistan?"

answer(question, chunks, features, pipe, NN, q_tokenizer, q_model)

Q: What is the сapital of Uzbekistan?
A: Tashkent

Based on:
and Alaska started calling them bufriedos, and then everyone did, and then finally Maureen officially changed the name." He paused. "I don't know what to do, Miles." "Yeah. I know." "I finished memorizing the capitals," he said. "Of the states?" "No. That was fifth grade. Of the countries. Name a country." "Canada," I said. "Something hard." "Urn. Uzbekistan?" "Tashkent." He didn't even take a moment to think. It was just there, at the tip of his tongue, as if he'd been waiting for me to say "Uzbekistan" all along. "Let's smoke." We walked to the bathroom and turned on the
so little, and I grabbed it tight, his cold seeping into me and my warmth into him. "I memorized the populations," he said. "Uzbekistan." "Twenty-four million seven hundred fifty-five thousand five hundred and nineteen." "Cameroon," I said, but it was too late. He was asleep, his hand limp in mine. I placed it back under the quilt and climbed up into his be

In [17]:
#Wrong
question = "What is the сapital of Great Britain?"

answer(question, chunks, features, pipe, NN, q_tokenizer, q_model)

Q: What is the сapital of Great Britain?
A: a general named Sedgwick said, 'They couldn't hit an elephant from

Based on:
and Alaska started calling them bufriedos, and then everyone did, and then finally Maureen officially changed the name." He paused. "I don't know what to do, Miles." "Yeah. I know." "I finished memorizing the capitals," he said. "Of the states?" "No. That was fifth grade. Of the countries. Name a country." "Canada," I said. "Something hard." "Urn. Uzbekistan?" "Tashkent." He didn't even take a moment to think. It was just there, at the tip of his tongue, as if he'd been waiting for me to say "Uzbekistan" all along. "Let's smoke." We walked to the bathroom and turned on the
ultimate nobility: King of Kings. Class over. You can pick up a copy of your final exam on the way out. Stay dry." It wasn't until I stood up to leave that I noticed Alaska had skipped class — how could she skip the only class worth attending? I grabbed a copy of the final for her. The final exam: